In [ ]:
from ratdata import data_manager as dm, process, ingest
from pathlib import Path
import numpy as np

In [ ]:
dm.db_connect('rat_data.db')
outfilename = 'data/teed.csv'

In [ ]:
# Get recording files from the database
open_loop_files_query = dm.RecordingFile.select().join(dm.StimSettings).where((dm.StimSettings.stim_type == 'continuous'))
on_off_files_query = dm.RecordingFile.select().join(dm.StimSettings).where((dm.StimSettings.stim_type == 'on-off'))
random_files_query = dm.RecordingFile.select().join(dm.StimSettings).where((dm.StimSettings.stim_type == 'random'))

In [ ]:
# Calculate TEED for each file with continuous stimulation
with open(outfilename, 'w') as outfile:
    for file in open_loop_files_query:
        amplitude = dm.StimSettings.get(dm.StimSettings.recording_file == file).max_amplitude
        teed = process.compute_teed_continuous_stim(amplitude, pulse_width=80, stim_frequency=130, impedance=1)
        line = '%s, %f' % (file.filename, teed) 
        print(line)
        outfile.write(line + '\n')
    

In [ ]:
# Calculate TEED for each file with ON/OFF stim
pulse_width = 80
f_stimulation = 130
with open(outfilename, 'a') as outfile:
    for file in on_off_files_query:
        amplitude = dm.StimSettings.get(dm.StimSettings.recording_file == file).max_amplitude
        recording = ingest.read_mce_matlab_file(str(Path(file.dirname) / file.filename))
        amplitude_data = process.generate_amplitude_from_stim_periods(recording, amplitude)
        if dm.is_recording_sliced(file.filename):
            fs = int(1 / recording.dt)
            slice = file.slice.get()
            amplitude_data, tt = process.trim_recording(amplitude_data, fs, slice.start, slice.length)
        else:
            tt = np.linspace(0, recording.electrode_data.shape[1] * recording.dt, recording.electrode_data.shape[1])
        teed = process.compute_teed_from_amplitude_recording(amplitude_data, tt, pulse_width, f_stimulation, 1)
        line = '%s, %f' % (file.filename, teed)
        print(line)
        outfile.write(line + '\n')
    

In [ ]:
# Calculate TEED for each file with random stim
pulse_width = 80
f_stimulation = 130
with open(outfilename, 'a') as outfile:
    for file in random_files_query:
        amplitude = dm.StimSettings.get(dm.StimSettings.recording_file == file).max_amplitude
        recording = ingest.read_mce_matlab_file(str(Path(file.dirname) / file.filename))
        amplitude_data = process.generate_amplitude_from_stim_periods(recording, amplitude)
        if dm.is_recording_rejected(file.filename):
            fs = int(1 / recording.dt)
            slice = file.slice.get()
            amplitude_data, tt = process.trim_recording(amplitude_data, fs, slice.start, slice.length)
        else:
            tt = np.linspace(0, recording.electrode_data.shape[1] * recording.dt, recording.electrode_data.shape[1])
        teed = process.compute_teed_from_amplitude_recording(amplitude_data, tt, pulse_width, f_stimulation, 1)
        line = '%s, %f' % (file.filename, teed) 
        print(line)
        outfile.write(line + '\n')

In [ ]:
pulse_width = 80
f_stimulation = 130
gui_datadir = Path('data/gui_recordings')

with open(outfilename, 'a') as outfile:
    for f in gui_datadir.iterdir():
        f_query = dm.GuiAmplitude.select().where(dm.GuiAmplitude.file == f.name)
        if f_query.count() == 1:
            rfile = f_query.get()
            stim = dm.StimSettings.select().join(dm.RecordingFile).where(dm.StimSettings.recording_file == rfile.recording).get()
            amplitude = stim.max_amplitude
            stim_data = ingest.read_stim_amplitude_from_gui_recording(gui_datadir / rfile.file, amplitude)
            tt = np.linspace(0, len(stim_data) / 500, len(stim_data))
            teed = process.compute_teed_from_amplitude_recording(stim_data, tt, pulse_width, f_stimulation, 1)
            line = '%s, %f' % (rfile.recording.filename, teed) 
            print(line)
            outfile.write(line + '\n')

In [ ]:
with open('data/teed.csv', 'r') as f:
    with open('data/teed_full.csv', 'w') as outfile:
        for line in f:
            line = line.strip(' \r\t\n')
            filename, teed = line.split(',')
            record = dm.RecordingFile.select().join(dm.Rat).where(dm.RecordingFile.filename == filename).get()
            stim_settings = dm.StimSettings.select().join(dm.RecordingFile).where(dm.RecordingFile.filename == filename).get()
            outline = '%s, %s, %s, %s, %s, %s, %s' % (filename, record.condition, record.rat.full_label, record.rat.group,
                                                    stim_settings.stim_type, stim_settings.max_amplitude, teed)
            print(outline)
            outfile.write(outline + '\n')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data/teed_full.csv', sep=',')

df.columns = ['filename', 'condition', 'rat', 'rat_group', 'stim_type', 'max_amplitude', 'power']
df = df.explode('power')
df['power'] = df['power'].astype('float') / 1e6

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='stim_type', y='power', data=df, palette='Dark2', boxprops=dict(alpha=.8))
sns.swarmplot(x='stim_type', y='power', data=df, s=4, palette='Dark2')
plt.xlabel('Stimulation type')
plt.ylabel('TEED in 1s [pW/$\Omega$]')
plt.title('Average energy delivered in 1s of stimulation')
plt.savefig('plots/teed.png', bbox_inches='tight', facecolor='white')